## Wind data cleaning

In [1]:
import pandas as pd
import numpy as np

In [11]:
# read wind data
wind = pd.read_excel('15_20_wind_data.xlsx')
wind.columns = wind.iloc[0]
wind = wind[2:]
wind.head()

,NaN,002092.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,000963.SZ,...,NaN,NaN,NaN,NaN,300134.SZ,NaN,NaN,NaN,NaN,NaN
2,Date,roe,mkt_cap,roa,roic,pe,esg_rating,NaN,Date,roe,...,pe,esg_rating,NaN,Date,roe,mkt_cap,roa,roic,pe,esg_rating
3,2015-01-01 00:00:00,NaN,107.744,NaN,NaN,1403.57,NaN,NaN,2015-01-01 00:00:00,NaN,...,1333.46,NaN,NaN,2015-01-01 00:00:00,NaN,142.195,NaN,NaN,148.066,NaN
4,2015-01-02 00:00:00,NaN,107.744,NaN,NaN,1403.57,NaN,NaN,2015-01-02 00:00:00,NaN,...,1333.46,NaN,NaN,2015-01-02 00:00:00,NaN,142.195,NaN,NaN,148.066,NaN
5,2015-01-03 00:00:00,NaN,107.744,NaN,NaN,1403.57,NaN,NaN,2015-01-03 00:00:00,NaN,...,1333.46,NaN,NaN,2015-01-03 00:00:00,NaN,142.195,NaN,NaN,148.066,NaN
6,2015-01-04 00:00:00,NaN,107.744,NaN,NaN,1403.57,NaN,NaN,2015-01-04 00:00:00,NaN,...,1333.46,NaN,NaN,2015-01-04 00:00:00,NaN,142.195,NaN,NaN,148.066,NaN


In [163]:
# read RepRisk data
full_code_rrid = pd.read_excel('384_code_rrid.xlsx')[['code','RepRisk_id']]
code_rrid_dict = {}
for i in range(len(full_code_rrid)):
    c = full_code_rrid.iloc[i]['code']
    r = full_code_rrid.iloc[i]['RepRisk_id']
    code_rrid_dict[r] = c
rr = pd.read_excel('rr_data_379_1520.xlsx')
rr.columns = ['RepRisk_ID', 'Date', 'RRR', 'E_Percentage', 'S_Percentage', 'G_Percentage']
tmp_rrid = rr['RepRisk_ID'].tolist()
tmp_code = []
for i in tmp_rrid:
    tmp_code.append(code_rrid_dict[i])
rr['code'] = tmp_code
rr.drop(columns = ['RepRisk_ID'], inplace=True)
rr.set_index(['code','Date'], inplace=True)
rr.head()

RRR  E_Percentage  S_Percentage  G_Percentage
code      Date                                                    
002092.SZ 2015-01-31  BB           0.0           0.0           0.0
          2015-02-28  BB           0.0           0.0           0.0
          2015-03-31  BB           0.0           0.0           0.0
          2015-04-30  BB           0.0           0.0           0.0
          2015-05-31  BB           0.0           0.0           0.0

#### RepRisk has only found 377 out of 379 companies that Wind has, so the two companies that don't have RepRisk data need to be dropped before joining two DFs.

In [119]:
rr_code = set([i[1] for i in rr.index])
print('There are {} companies in RepRisk dataset'.format(len(rr_code)))
diff = list(set(all_codes) - rr_code)
print('The difference is {}'.format(diff)) 

There are 377 companies in RepRisk dataset
The difference is ['601318.SH', '000898.SZ']


In [161]:
# clean wind data
wind_df = wind.iloc[:,0:7]
new_header = wind_df.iloc[0]
code = wind_df.columns[1]
all_codes = []
all_codes.append(code)
wind_df = wind_df[1:]
wind_df.columns = new_header.values
wind_df['code'] = code
wind_df.set_index(['code','Date'],inplace = True)
for i in range(8,len(wind.columns),8):
    tmp_df = wind.iloc[:,i:i+7]
    code = tmp_df.columns[1]
    if code in diff:
        continue
    else:
        all_codes.append(code)
        new_header = tmp_df.iloc[0]
        tmp_df = tmp_df[1:]
        tmp_df.columns = new_header.values
        tmp_df['code'] = code
        tmp_df.set_index(['code','Date'],inplace = True)
        wind_df = pd.concat([wind_df,tmp_df])

#### Now wind_df has two indexes, date (2265 days from 2015/01/01 to 2021/03/14) and company code (377 Chinese A-share companies) 

In [162]:
wind_df

roe  mkt_cap  roa roic       pe esg_rating
code      Date                                                  
002092.SZ 2015-01-01  NaN  107.744  NaN  NaN  1403.57        NaN
          2015-01-02  NaN  107.744  NaN  NaN  1403.57        NaN
          2015-01-03  NaN  107.744  NaN  NaN  1403.57        NaN
          2015-01-04  NaN  107.744  NaN  NaN  1403.57        NaN
          2015-01-05  NaN  113.165  NaN  NaN   1474.2        NaN
...                   ...      ...  ...  ...      ...        ...
300134.SZ 2021-03-10  NaN  75.0613  NaN  NaN      NaN         B-
          2021-03-11  NaN  75.9823  NaN  NaN      NaN         B-
          2021-03-12  NaN  74.6776  NaN  NaN      NaN         B-
          2021-03-13  NaN  74.6776  NaN  NaN      NaN         B-
          2021-03-14  NaN  74.6776  NaN  NaN      NaN         B-

[853905 rows x 6 columns]

## RepRisk data cleaning

In [187]:
full_df = wind_df.join(rr, how='outer')

# we can fill the missing ESG data with the latest rating (ffill) 
for col in ['esg_rating', 'RRR', 'E_Percentage', 'S_Percentage', 'G_Percentage']:
    full_df[col] = full_df.groupby(level=0)[col].ffill()
    
# then for the several missing cells at the top of the dataframe, use the nearest future rating to fill (bfill) 
for col in ['esg_rating', 'RRR', 'E_Percentage', 'S_Percentage', 'G_Percentage']:
    full_df[col] = full_df.groupby(level=0)[col].bfill()

In [188]:
full_df

roe  mkt_cap  roa roic       pe esg_rating  RRR  \
code      Date                                                          
000001.SZ 2015-01-01  NaN   1809.7  NaN  NaN  8.27671         B-  CCC   
          2015-01-02  NaN   1809.7  NaN  NaN  8.27671         B-  CCC   
          2015-01-03  NaN   1809.7  NaN  NaN  8.27671         B-  CCC   
          2015-01-04  NaN   1809.7  NaN  NaN  8.27671         B-  CCC   
          2015-01-05  NaN  1830.27  NaN  NaN  8.37077         B-  CCC   
...                   ...      ...  ...  ...      ...        ...  ...   
603885.SH 2021-03-10  NaN  269.165  NaN  NaN      NaN         B-   BB   
          2021-03-11  NaN  294.725  NaN  NaN      NaN         B-   BB   
          2021-03-12  NaN  293.545  NaN  NaN      NaN         B-   BB   
          2021-03-13  NaN  293.545  NaN  NaN      NaN         B-   BB   
          2021-03-14  NaN  293.545  NaN  NaN      NaN         B-   BB   

                      E_Percentage  S_Percentage  G_Percentage  
code      Date                                                  
000001.SZ 2015-01-01           0.0           0.0           1.0  
          2015-01-02           0.0           0.0           1.0  
          2015-01-03           0.0           0.0           1.0  
          2015-01-04           0.0           0.0           1.0  
          2015-01-05           0.0           0.0           1.0  
...                            ...           ...           ...  
603885.SH 2021-03-10           1.0           0.0           0.0  
          2021-03-11           1.0           0.0           0.0  
          2021-03-12           1.0           0.0           0.0  
          2021-03-13           1.0           0.0           0.0  
          2021-03-14           1.0           0.0           0.0  

[853905 rows x 10 columns]

## Finally, map letter ratings to integers (for further analysis)

In [189]:
dict_syntao = {'D':1,'C-':2,'C':3,'C+':4,'B-':5,'B':6,'B+':7,'A-':8,'A':9,'A+':10}
dict_rr = {'AAA':10,'AA':9,'A':8,'BBB':7,'BB':6,'B':5,'CCC':4,'CC':3,'C':2,'D':1}
# working multiindex is tricky, to alter the letter values as int, we reset the index as default and use .at to change values
int_tot_df = full_df.reset_index()

In [195]:
for i in range(len(int_tot_df)):
    st_letter = int_tot_df['esg_rating'][i]
    rr_letter = int_tot_df['RRR'][i]
    int_tot_df.at[i,'esg_rating'] = dict_syntao[st_letter]
    int_tot_df.at[i,'RRR'] = dict_rr[rr_letter]

In [198]:
full_df = int_tot_df.set_index(['code','Date'])
full_df

roe  mkt_cap  roa roic       pe esg_rating RRR  \
code      Date                                                         
000001.SZ 2015-01-01  NaN   1809.7  NaN  NaN  8.27671          5   4   
          2015-01-02  NaN   1809.7  NaN  NaN  8.27671          5   4   
          2015-01-03  NaN   1809.7  NaN  NaN  8.27671          5   4   
          2015-01-04  NaN   1809.7  NaN  NaN  8.27671          5   4   
          2015-01-05  NaN  1830.27  NaN  NaN  8.37077          5   4   
...                   ...      ...  ...  ...      ...        ...  ..   
603885.SH 2021-03-10  NaN  269.165  NaN  NaN      NaN          5   6   
          2021-03-11  NaN  294.725  NaN  NaN      NaN          5   6   
          2021-03-12  NaN  293.545  NaN  NaN      NaN          5   6   
          2021-03-13  NaN  293.545  NaN  NaN      NaN          5   6   
          2021-03-14  NaN  293.545  NaN  NaN      NaN          5   6   

                      E_Percentage  S_Percentage  G_Percentage  
code      Date                                                  
000001.SZ 2015-01-01           0.0           0.0           1.0  
          2015-01-02           0.0           0.0           1.0  
          2015-01-03           0.0           0.0           1.0  
          2015-01-04           0.0           0.0           1.0  
          2015-01-05           0.0           0.0           1.0  
...                            ...           ...           ...  
603885.SH 2021-03-10           1.0           0.0           0.0  
          2021-03-11           1.0           0.0           0.0  
          2021-03-12           1.0           0.0           0.0  
          2021-03-13           1.0           0.0           0.0  
          2021-03-14           1.0           0.0           0.0  

[853905 rows x 10 columns]

Above is the panel data<br> <b>Time</b>: 2015/01/01 - 2021/03/14 <br> <b>Company coverage</b>: 377 Chinese A share companies, included in both RepRisk and Syntao <br> <b>CFP indicators</b>: ROE, ROA, ROIC, PE, Market capitalization

In [199]:
full_df.to_csv('panel_data.csv')